## Decision Trees

Here I am and make a Regression decision tree class with will work on features with binary values


### Splitting

In the general regression case for each region $R$ we seek the splitting variable j and splitting point s that solve (ESL 9.13), and partitions the space into two regions $R_1, R_2$ with the lowest total squared error. 

$$ \min_{j,s} \left[ 
\min_{c_1} \sum_{x_i \in R_1(j,s)} (y_i-c_1)^2 + 
\min_{c_2} \sum_{x_i \in R_2(j,s)} (y_i-c_2)^2 
\right]$$

Where $c_1, c_2$ are the average $y_i$ in the child regions. 

In the classification case minimize use weighted node impurity, Gini or cross entropy, to choose the split.

For the regression case with binary features, there is only one possible split point $s$ for each feature, simplifying the search to:

$$ \min_{j} \left[ 
\min_{c_1} \sum_{x_i \in R_1(j)} (y_i-c_1)^2 + 
\min_{c_2} \sum_{x_i \in R_2(j)} (y_i-c_2)^2 
\right]$$

The *search* function iterates over all current nodes and will choose the split that have the biggest improvement in the squared error

$$ I = \left\vert err(R_1) + err(R_2) - err(R) \right\vert$$


### Data Structures

In mathematical notation, a decicion tree is defined by the splitting variables and points. This is one view of the tree, and if augmented with averages (or modes in classification) at the terminal nodes it is sufficent for prediction. 

The main question for the above data structure is "How does an observation fall down the tree". This is not all that easy. I'm thinking of a dictionary keyed on integers of node number. The value it stores is another dictionary with members below. 

There will be parent nodes that have splits, and there will be terminal nodes that have predictions. the new format will be

- terminal : bool
- prediction : float
- left : int
- right : int
- split_feature : int

However, during training I need to store membership of the training observations in each node so I can calcualte loss or improvement when greedily seeking splits. 

This could be done by appending a column to the dataframe or numpy array and storing the integer node membership there. Filtering will then provide access to the observations of the node. 

### Tuning Paramaters

There is only one tuning paramater for the tree and it determins when we stop growing. One flavor is the number of observations in each node before stopping, another is the complexity paramater at which we stop splitting. 

Alternativly we can grow a large tree and then prune it down with cross validation.

After getting the basics done (fitting a tree to m in each terminal node I will come back to this idea

### Data Generation

I'd like to start with a simple model with generated data

$$y = \beta_1 z_1 + \beta_2 z_2 + \beta_3 (z_1 \cdot z_2) $$

In [3]:
import numpy as np
from typing import Dict, Tuple, Generic, TypeVar

In [4]:
##Generate Data

#set seed
np.random.seed(2020)

#generate data
n = 100
beta = np.random.uniform(0,1,3)
print("beta",beta)
X = np.random.randint(0,2,(n,2))
y = X[:,0]* beta[0] + X[:,1] * beta[1] + X[:,0] * X[:,1] * beta[2]

#check dimensions
assert(len(y) == X.shape[0])

beta [0.98627683 0.87339195 0.50974552]


In [5]:
#set up typing
T = TypeVar('T')
SplitDict = Dict[ str, T ]
NodeDict = Dict[ int, SplitDict ]

In [92]:
class Decision_Tree:
    
    m : int # minimum of obs in terminal nodes
    theta : NodeDict = dict() #history of node splits in tree
    
    def __init__(self, m = 5):
        self.m = m;
    
    def loss(self, y, ybar):
        """ squared error loss
        """
        return np.sum( np.power(y-ybar,2) )
    
    def node_split(self,X : np.ndarray ,y : np.ndarray) -> Tuple[ int, float, bool ]:
        """ given a set of observations, finds the feature which produces the minimum split
            respects the minimum observation terminal node restriction, will not split 
            returns:
                int : index of feature with lowest total loss split
                float : total loss after splitting 
                bool: If these is any valid split given 
        """
        
        #check dimensions
        assert( len(y) == X.shape[0])
        
        #iterate over all features and try every split:
        min_loss = None
        j_split = None
        valid = False
        for f in range(X.shape[1]): #for each feature
            #region 1
            y1 = y[X[:,f] == 0]
            y1bar = np.mean(y1)
            l1 = self.loss(y1,y1bar)
            r1size = len(y[X[:,f] == 0])
            #region 2
            y2 = y[X[:,f] == 1]
            y2bar = np.mean(y2)
            l2 = self.loss(y2,y2bar)
            r2size = len(y[X[:,f] == 0])
            
            #check if loss decreases, and that does not split beyond min size
            if ((min_loss is None) or (l1+l2 < min_loss)) and (r2size >= self.m and r1size >= self.m):
                min_loss = l1+l2
                j_split = f
                valid = True
            
            print("feature {}, total : {:.2f}, left : {:.2f}, right : {:.2f}".format(f,l1+l2, l1,l2 ))
        
        return (j_split, min_loss, valid)
    
    def search(self, 
               X : np.ndarray,
               y : np.ndarray,
               mem : np.ndarray
              ) -> Tuple[ int, int, bool]:
        """ Iterates over all nodes as defined by mem the node membership array 
            find the node with the largest improvement I
            returns:
                int : index of node to split
                int : index of feature to split
                bool : If there is any valid input 
        """
        
        #set up variables
        max_improvement = None
        node_to_split = None
        index_of_feature = None
        valid = False
        
        #loop through current terminal nodes
        for node in set(mem):
            
            #get best split for node
            split_feature, split_loss, split_valid = self.node_split( X[mem == node], y[mem == node] )
            if (split_valid == True):
                loss_before = self.loss( y[mem == node], np.mean(y[mem == node]) )
                improvement  = np.abs( loss_before - split_loss )
            
            #check if better and store
            if ((split_valid == True) and (max_improvement == None or improvement > max_improvement) ):
                valid = split_valid
                max_improvement = improvement
                node_to_split = node
                index_of_feature = split_feature
            
            #check if there is any actual improvement
            if np.isclose(max_improvement,0):
                valid = False
        
        return [ node_to_split, index_of_feature, valid  ]  
        

    def train(self, 
              X : np.ndarray,
              y : np.ndarray
             ) -> None:
        
        #generate node membership vector
        assert(len(y) == X.shape[0])
        n, p = X.shape
        mem = np.ones(n)
        
        #loop until fully grown
        growing = True
        split_count = 0
        max_splits = 10
        while growing:
            
            #get best split for current terminal nodes
            node_to_split, index_of_feature, valid = self.search(X,y,mem)
            
            #exit if no valid split
            if not valid:
                growing = False
                break
            
            #get obeservation index to update
            row_index = np.argwhere(mem == node_to_split)
            
            #for each observation choose the next node
            #0 goes left, 1 goes right
            next_node = int(np.max(mem) + 1)
            left = row_index[ X[row_index,index_of_feature] == 0 ]
            right = row_index[ X[row_index,index_of_feature] == 1 ]
            
            #update
            mem[left] = next_node
            mem[right] = next_node + 1
            
          
            #add intermediate node to the dictionary with splitting information
            #could add prediction if desired
            self.theta[node_to_split] = {'terminal' : False,
                                    'prediction' : None,
                                    'left' : next_node,
                                    'right' : next_node + 1,
                                    'split_feature' : index_of_feature                
                                    }
            
            
            split_count+=1
            if split_count >= max_splits:
                growing = False
            print("itr {} done".format(split_count))
            #print(mem)
        
        #add terminal nodes to dictionary with predictions
        for terminal_node in set(mem):
            self.theta[terminal_node] = {'terminal' : True,
                                        'prediction' : np.mean( y[ np.argwhere(mem == terminal_node) ] ),
                                        'left' : None,
                                        'right' : None,
                                        'split_feature' : None                
                                        }
            #print(self.theta[terminal_node])
    
    def get_prediction(self, x : np.ndarray) -> float:
        """ sends a single observation down the regression tree, returns prediction
        """
        next_node = 1
        terminal = False
        while not terminal:
            #get split info
            info = self.theta[next_node]
            #return pred if at terminal node
            if info['terminal'] == True:
                return info['prediction']
            #otherwise find which way observation goes
            if x[ info["split_feature"]] == 0 :
                next_node = info['left']
            else:
                next_node = info['right']
        
    
    def predict(self, X : np.ndarray) -> np.ndarray:
        """ use the trained tree to predict for observations in X
            no checking for correct dimensions as trained
            return:
                np.ndarray: predicted values for observations
        """
        
        return np.array([ self.get_prediction(row) for row in X ])
        
            
                

Next Step:
- store something as growing that can be used to predidct
- predict. 

In [93]:
dt = Decision_Tree()
dt.train(X,y)

feature 0, total : 34.44, left : 8.76, right : 25.68
feature 1, total : 39.57, left : 12.17, right : 27.41
itr 1 done
feature 0, total : 8.76, left : 8.76, right : 0.00
feature 1, total : 0.00, left : 0.00, right : 0.00
feature 0, total : 25.68, left : 0.00, right : 25.68
feature 1, total : 0.00, left : 0.00, right : 0.00
itr 2 done
feature 0, total : 8.76, left : 8.76, right : 0.00
feature 1, total : 0.00, left : 0.00, right : 0.00
feature 0, total : 0.00, left : 0.00, right : 0.00
feature 1, total : 0.00, left : 0.00, right : 0.00
feature 0, total : 0.00, left : 0.00, right : 0.00
feature 1, total : 0.00, left : 0.00, right : 0.00
itr 3 done
feature 0, total : 0.00, left : 0.00, right : 0.00
feature 1, total : 0.00, left : 0.00, right : 0.00
feature 0, total : 0.00, left : 0.00, right : 0.00
feature 1, total : 0.00, left : 0.00, right : 0.00
feature 0, total : 0.00, left : 0.00, right : 0.00
feature 1, total : 0.00, left : 0.00, right : 0.00
feature 0, total : 0.00, left : 0.00, righ

In [99]:
pred = dt.predict(X)
loss = dt.loss(pred,y)
print("training set loss",loss)

training set loss 1.4298103907130839e-30
